Libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader

Graph Convolutional Network

In [2]:
# Define a simple GCN model
class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Apply the first GCN layer
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)

        # Apply the second GCN layer
        x = self.conv2(x, edge_index)

        return x

Load Dataset

In [3]:
# Load the Cora dataset
dataset = Planetoid(root='./data', name='Cora')

# Create a data loader
loader = DataLoader(dataset, batch_size=32)

Instantiate Model

In [4]:
# Create the model
model = GCN(dataset.num_features, 64, dataset.num_classes)

# Define the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss() #define loss function 

Train Model

In [5]:
# Train the model
for epoch in range(200):
    for data in loader:
        model.train()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data.y)
        loss.backward()
        optimizer.step()

Evaluate Model

In [6]:
# Evaluate the model
model.eval()
with torch.no_grad():
    correct = 0
    for data in loader:
        output = model(data)
        pred = output.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    accuracy = correct / len(data)
    print("Accuracy:", accuracy)

Accuracy: 2692.0


Visualization